<a href="https://colab.research.google.com/github/shubhigupta991/Reddit-Flair-Detection/blob/main/scripts/WebScrapping%20and%20PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collecting the data from reddit

We are collecting the data with the help of PRAW

PRAW stands for Python Reddit API Wrapper.

In [1]:
!pip install praw

     |████████████████████████████████| 153kB 12.4MB/s 
     |████████████████████████████████| 204kB 19.6MB/s 


In [2]:
import praw
import pandas as pd
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords
import datetime as dt
nltk.download('all')
from bs4 import BeautifulSoup

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [3]:
reddit = praw.Reddit(client_id='KqCyLYQgMNwp4w', client_secret='cA9UCAPiVadgs4FTsnZ3RqJUR0hROw', user_agent='Flair-Detector', 
                     username='shubhigupta09', password='flair123')

In [4]:
subreddit = reddit.subreddit('india')

In [5]:
labels = {"title":[], "id":[], "score":[], "url":[], "created": [], "num_of_comments": [], "body":[], "author":[], "comments":[], "flair":[]}
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", 
          "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA","Coronavirus"]

In [ ]:
for flair in flairs:
  
  get_subreddits = subreddit.search(flair, limit=100)
  
  for each_post in get_subreddits:
    
    labels["flair"].append(flair)
    labels["title"].append(each_post.title)
    labels["score"].append(each_post.score)
    labels["id"].append(each_post.id)
    labels["url"].append(each_post.url)
    labels["num_of_comments"].append(each_post.num_comments)
    labels["created"].append(each_post.created)
    labels["body"].append(each_post.selftext)
    labels["author"].append(each_post.author)
    
    each_post.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in each_post.comments:
      comment = comment + ' ' + top_level_comment.body
    labels["comments"].append(comment)

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [ ]:
data = pd.DataFrame(labels_dict)
time =data["created"].apply(get_date)
data =data.assign(timestamp = time)
del data['created']
data.to_csv('reddit-india-data.csv', index=False)

In [ ]:
data=pd.read_csv('reddit-india-data.csv')
data.head()

In [ ]:
def string(value):
    return str(value)

In [ ]:
data['title'] = string(data['title'])
data['body'] = string( data['body'])
data['comments'] = string(data['comments'])

In [ ]:
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
stopWords = set(stopwords.words('english'))
def text_cleaning(text):
   
    text = BeautifulSoup(text, "lxml").text
    text = text.lower()
    text = replace_by_space.sub(' ', text)
    text = bad_symbols.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopWords)
    return text

In [ ]:
data['title'] = text_cleaning(data['title'])
data['body'] = text_cleaning( data['body'])
data['comments'] = text_cleaning(data['comments'])

In [ ]:
combined_features = data["title"] + data["comments"] + data["url"] + data["body"]
data = data.assign(combined_features = combined_features)

In [ ]:
data.to_csv('data.csv')

In [ ]:
pd.read_csv('data.csv')